# Measure contributions of parts of OCR pipeline

## Possible OCR pipelines (actions are green)

![ocr_flow](assets/ocr_flow.drawio.jpg)

#### The are 4 groups of actions in the full pipeline (green boxes). The purpose of this notebook is to test if all of these actions really helps with the OCR results, and if they do, by how much.

- **Find the single best ensemble.**
- Which of the 4 image processing pipelines improve OCR performance?
- Two OCR engines: `Tesseract` & `EasyOCR`. `Tesseract` is the current leader in open source OCR engines, does adding `EasyOCR` improve the results?
- The `combine text` function is only needed if we stick with the ensemble approach. I.e. only if we use more than one image processing pipeline or more than one OCR engine.
- The `clean text` function corrects misspellings and common OCR errors with punctuation, spacing, etc. We want to measure its efficacy.

## Comparison strategy

#### We're doing ablations on the OCR pipeline.
- How well do `Tesseract` and `EasyOCR` perform on their own without image pre-processing. I'll also try the engine directly grafted to the `clean text` function.
- How well do each of the image pre-processing steps help the OCR process? and which ones work well with which OCR engine. I'm going to try various permutations of these.
- Can I whittle this down to one or zero image pre-processing pipelines and one OCR engine? If so, then this would allow me to drop the `combine text` step.
- How much does the `clean text` step help?
- Note that `EasyOCR` uses a fair bit of GPU resources and if we can remove it it will speed up the OCR pipeline significantly. I.e. `EasyOCR` is difficult to parallelize.

#### Scoring
- I'll use an expert derived gold standard to compare against the ablation sequences.
- I am using Levenshtein distance as the scores. Levenshtein distance counts character mismatches between sequences in a best case pairwise alignment.

## Setup

In [1]:
import json
from IPython.display import display, HTML
from collections import defaultdict, namedtuple
from colorama import Back, Fore, Style
from itertools import groupby
from pathlib import Path
from types import SimpleNamespace

from digi_leap.pylib import consts
from digi_leap.pylib.ocr import ocr_compare as compare

In [2]:
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [3]:
ARGS = SimpleNamespace(
    database=Path(consts.DATA_DIR / "sernec" / "sernec.sqlite"),
    gold_set="test_gold_set",
    score_set="notebook_scores",
    notes="",
    csv_path=consts.DATA_DIR / "sernec" / "gold_std_2022-06-28_sample.csv",
)

## Gold standard

In [4]:
# Save a new gold standard to a database

# comp.insert_gold_std(ARGS.csv_path, ARGS.database, ARGS.gold_set)

In [5]:
# Read a gold standard for the database

GOLD_STD = compare.select_gold_std(ARGS.database, ARGS.gold_set)
GOLD_DICT = {g["gold_id"]: g for g in GOLD_STD}

## OCR scores

In [6]:
scorer = compare.Scorer(ARGS)

In [7]:
# Calculate new scores

# SCORES = scorer.calculate(GOLD_STD)
# scorer.insert_scores(SCORES)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:36<00:00, 24.05s/it]


In [8]:
SCORES = scorer.select_scores()

## Examine scores

In [9]:
def peek_scores(scores):
    grouped_scores = groupby(scores, key=lambda s: s["label_id"])

    for labels_id, scores in grouped_scores:
        print("=" * 80)
        print(labels_id)
        scores = sorted(
            scores, key=lambda s: (s["score"], len(json.loads(s["actions"])))
        )
        for score in scores:
            print(f"{score['score']:4d}  {score['actions']}")


# peek_scores(SCORES)

In [10]:
def msa_top_scores(scores, gold_std, line_align):
    grouped_scores = groupby(scores, key=lambda s: s["label_id"])

    for label_id, scores in grouped_scores:
        scores = list(scores)
        gold = gold_std[scores[0]["gold_id"]]
        min_score = min(s["score"] for s in scores)
        top = [gold["gold_text"]]
        top += [s["score_text"] for s in scores if s["score"] == min_score]
        top = [" ".join(ln.split()) for ln in top]
        aligned = line_align.align(top)

        print(f"{label_id}  {min_score}")

        rows = len(aligned)
        cols = len(aligned[0])
        colored = [list(a) for a in aligned]

        for col in range(cols):
            col_chars = [aligned[row][col] for row in range(rows)]
            if len(set(col_chars)) > 1:
                for row in range(rows):
                    colored[row][col] = (
                        Back.LIGHTGREEN_EX
                        + Fore.WHITE
                        + colored[row][col]
                        + Style.RESET_ALL
                    )

        for ln in colored:
            print("".join(ln))
        print()


msa_top_scores(SCORES, GOLD_DICT, scorer.line_align)

71341  7
FLORA OF ALABAMA Pike County No. 15 Botanical Name Iris virginica L. Family Iridaceae Locality Pocosin 6 mi. E. Troy Ala. Col. by J⋄. Popham Det. by J. Popham Date 3-29-67 Remarks: Herbarium of Troy State College
FLORA OF ALABAMA Pike County No. 15 Botanical Name Iris virginica L. Family Iridaceae Locality Pocosin 6 mi. E. Troy Ala. Col. by J+. Popham Det. by L⋄⋄⋄opha⋄ Date 3=29-67 Remarks: Herbarium of Troy State College

71540  4
HERBARIUM OF NORTHEAST LOUISIANA UNIVERSITY, MONROE FLORA OF LOUISIANA Parish: RAPIDES Solidago altissima L. Along Lake Buhlow Airport Road and lake, off of US 165. Outside of Alexandria, La. Sec. 16, T2N, R1E. Collector: Steven Schutz, Christy Schutz No.: 1095 Date: 4 Sept. 1977
HERBARIUM OF NORTHEAST LOUISIANA UNIVERSITY, MONROE FLORA OF LOUISIANA Parish: RAPLDES Solidago altissima L. Along Lake Buhlow Airport Road and lake, off 0f US 165, Outside of Alexandria, La. Sec. 16, T2N, RLE. Collector: Steven Schutz, Christy Schutz No.: 1095 Date: 4 Sept

ValueError: One of 'h¼' these characters are missing from the substitution matrix.

In [ ]:
PipelineScore = namedtuple("PipelineScore", "score pipeline")


def scores_by_pipeline(scores, gold_std):
    tally = defaultdict(int)

    for score in scores:
        tally[score["actions"]] += score["score"]

    tally = [(v, len(a), a) for k, v in tally.items() if (a := json.loads(k))]
    tally = sorted(tally)
    return [PipelineScore(t[0], t[2]) for t in tally]


summed = scores_by_pipeline(SCORES, GOLD_DICT)

for sum_ in summed:
    print(sum_)